## 1. Retrieve




### Import packages

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
##This is so variables get printed within jupyter
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [4]:
##import packages
import os
import cdsapi ## check the current working directory, which should be the UNSEEN-open directory
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs

In [4]:
import sys

sys.path.append('../../')

In [5]:
from src.CDSretrieve import retrieve

In [6]:
##We want the working directory to be the UNSEEN-open directory
pwd = os.getcwd() ##current working directory is UNSEEN-open/Notebooks/1.Download
pwd #print the present working directory
os.chdir(pwd+'/../../') # Change the working directory to UNSEEN-open
os.getcwd() #print the working directory

'/lustre/soge1/projects/ls/personal/timo/UNSEEN-open/Notebooks/1.Download'

'/lustre/soge1/projects/ls/personal/timo/UNSEEN-open'

## Download all data

We will be using the SEAS5 hindcast, which is a dataset running from 1981-2016. The hindcast is initialized every month with 25 ensemble members and the forecast run for 6 months, indicated by blue horizontal bars below. February is forecasted by 6 initialization months (September-February). We discard the first month of the forecast because of dependence between the forecasts, explained in the evaluation section and are left with 5 initialization months (Sep-Jan) and 25 ensemble members forecasting february precipitation each year, totalling to an increase of 125 times the observed length.

For a summary of all available C3S seasonal hindcasts, their initialization months and more specifics, please see [ECMWF page](https://confluence.ecmwf.int/display/CKB/C3S+Seasonal+Forecasts) and the [SEAS5 paper](https://doi.org/10.5194/gmd-12-1087-2019).  



![title](../../graphs/Data_illustration.png)

The first download example above downloaded all 25 ensemble members for the forecast initialized in January (the bottom bar). We should repeat this over the other initialization month and over all years (1981-2016).

In [48]:
def get_init_months(target_month):
    target = target_month if isinstance(target_month, list) else [target_month]
    init_list = [target[0] - x for x in range(1,7 - len(target))]
    init_months = [12 + i if i < 1 else i for i in init_list]
    leadtimes = [np.arange(x, x + len(target)) for x in range(2,8 - len(target))]
    return(init_months, leadtimes)



In [49]:
get_init_months(target_month=[11,12,1])

([10, 9, 8], [array([2, 3, 4]), array([3, 4, 5]), array([4, 5, 6])])

Leadtimes should be [[2,3,4],[3,4,5],[4,5,6]]

In [44]:
# np.arange(1981, 2017)
def print_retrieve_arguments(target_months):
    target_months = target_months if isinstance(target_months, list) else [target_months]
    init_months, leadtimes = get_init_months(target_months)
    for j in range(2):  ##add if error still continue
        for i in range(len(init_months)):
            init_month = init_months[i]
            leadtime_months = leadtimes[i]
            if 12 in init_months:
                if init_month < 6:
                    year = years[j] + 1
                else:
                    year = years[j]
            else:
                year = years[j]
            print('year = ' + str(year) + ' init_month = ' + str(init_month) +
                  ' leadtime_month = ' + str(leadtime_months))

In [1]:
print_retrieve_arguments([3,4,5])

NameError: name 'print_retrieve_arguments' is not defined

For our download, we loop over initialization months and years. Because we only want February precipitation, the leadtime month (forecast length) changes with the initialization month. For example, in the September initialized forecasts, we only want the leadtime month 6 = February. For August initialized this is leadtime 5, etc. Furthermore, the year the forecast is initialized is required for the download. For September - December initialized forecasts, this is the target year-1. For January it is the same year as the target year. For example, for the first two target years this looks like the following:

Write a function that is used for the download.

And start the download! In total, we request 35 years x  initialization dates = 175 requests. 
I could try sending just 5 request of the different initialization dates for all years?

In [36]:
c = cdsapi.Client()
def retrieve_single(variables, year, init_month, leadtimes, area, folder):
    """Retrieve SEAS5 data from CDS.
        
        Parameters
        ----------
        variable : str
        year : int
            str(year) is used in the function
        init_month : Initialization month, integer. 
            For example, if the target month is February (2), initialization months are August-January (8-12,1) forecasting February
        leadtime_month : The lead times you want, integer. 
            Use of single months is much faster. The first lead time is 1 (Leadtime 0 does not exist).
            For example, for forecasting February from initialization month 1 (January), the leadtime months is 2. 
            For initialization month 12 (december), the lead time month is 3 (February).
        folder : The path to the folder where to store the data. 

        Returns
        -------
        Saves the files in the specified folder.
    """
    c.retrieve(
        'seasonal-monthly-single-levels',
        {
            'format': 'netcdf',
            'originating_centre': 'ecmwf',
            'system': '5',
            'variable': variables,
            'product_type': ['monthly_mean'], #'monthly_maximum',, 'monthly_standard_deviation',],
            'year': str(year),  #data before 1993 is available.  
            'month': "%.2i" % init_month,
            'leadtime_month': [str(a) for a in leadtimes],
            'area': area,
        },
        folder + str(year) + "%.2i" % init_month + '.nc')

In [40]:
def retrieve_SEAS5(variables, target_months, area, folder, years=np.arange(1981, 2017)):
    """Retrieve SEAS5 data from CDS.
        
        Parameters
        ----------
        variables : The variables to be downloaded, str. Can also be one variable.
        target_months : The month(s) of interest, int.
            For example, for JJA, use [6,7,8]. 
            Must be consecutive (e.g. no May, July, August).
        area : The domain to download the data over, [North, West, South, East,].
            For example, to dowload longitude 30,70 and latitude -10, 120, use [70, -11, 30, 120,]
        years : Defaults to the entire hindcast years from 1981-2016
        folder : The path to the folder where to store the data. 

        Returns
        -------
        Saves the files in the specified folder.
    """
    init_months, leadtimes = get_init_months(target_months)
    for j in range(len(years)):  ##add if error still continue
        for i in range(len(init_months)):
            init_month = init_months[i]
            leadtime_months = leadtimes[i]
            if 12 in init_months:
                if init_month < 6:
                    year = years[j]
                else:
                    year = years[j] - 1
            else:
                year = years[j]

            if not os.path.isfile(folder + str(year) + "%.2i" % init_month + '.nc'):
                retrieve_single(variables=variables,
                                year=year,
                                init_month=init_month,
                                leadtimes=leadtime_months,
                                area = area,
                                folder=folder)

The download sometimes fails. When redoing the request it does download. I don't know what is causing the failure? Below I donwload the file that failed. 

In [34]:
retrieve_SEAS5(variables = ['2m_temperature','2m_dewpoint_temperature'], target_months = [6,7,8], 
         area = [70, -11, 30, 120,], folder = '../Siberia_example/')

## ERA5 data download

ERA5 is downloaded using the monthly means for single levels `reanalysis-era5-single-levels-monthly-means`. 
The function takes the variables, target months, area and folder. The function downloads the data on a 1x1 degree grid, similar to the SEAS5 data. The default dataset is 0.25 x 0.25 (so higher resolution data is available). 


In [38]:
c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'netcdf',
        'product_type': 'monthly_averaged_reanalysis',
        'variable': [
            '2m_dewpoint_temperature', '2m_temperature',
        ],
        'area'          : [70, -11, 30, 120], # North, West, South, East.          Default: global
        'grid'          : [1.0, 1.0],       # Latitude/longitude grid.           Default: 0.25 x 0.25
        'year': [
            '1979', '1980', '1981',
            '1982', '1983', '1984',
            '1985', '1986', '1987',
            '1988', '1989', '1990',
            '1991', '1992', '1993',
            '1994', '1995', '1996',
            '1997', '1998', '1999',
            '2000', '2001', '2002',
            '2003', '2004', '2005',
            '2006', '2007', '2008',
            '2009', '2010', '2011',
            '2012', '2013', '2014',
            '2015', '2016', '2017',
            '2018', '2019', '2020',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': '00:00',
    },
    '../Siberia_example/ERA5_Siberia1x1.nc')

2020-08-31 10:48:48,744 INFO Welcome to the CDS
2020-08-31 10:48:48,746 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2020-08-31 10:48:54,251 INFO Request is queued
2020-08-31 10:48:55,267 INFO Request is running
2020-08-31 10:53:12,220 INFO Request is completed
2020-08-31 10:53:12,222 INFO Downloading http://136.156.133.36/cache-compute-0010/cache/data8/adaptor.mars.internal-1598867335.2534816-8321-21-66d0138a-814c-4c5b-8160-2aca9e46f31d.nc to ../Siberia_example/ERA5_Siberia1x1.nc (20.6M)
2020-08-31 10:53:13,111 INFO Download rate 23.2M/s  


Result(content_length=21608812,content_type=application/x-netcdf,location=http://136.156.133.36/cache-compute-0010/cache/data8/adaptor.mars.internal-1598867335.2534816-8321-21-66d0138a-814c-4c5b-8160-2aca9e46f31d.nc)